In [1]:
from collections import OrderedDict
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
from torchvision.models import resnet101
import torch.nn as nn
import os
import cv2
import random
import torch.utils.model_zoo as model_zoo
from copy import deepcopy

from matplotlib import cm
import scipy.misc as misc
import matplotlib
import imageio

In [2]:
from collections import OrderedDict

import torch
from torch import nn


class IntermediateLayerGetter(nn.ModuleDict):
    """
    Module wrapper that returns intermediate layers from a model
    It has a strong assumption that the modules have been registered
    into the model in the same order as they are used.
    This means that one should **not** reuse the same nn.Module
    twice in the forward if you want this to work.
    Additionally, it is only able to query submodules that are directly
    assigned to the model. So if `model` is passed, `model.feature1` can
    be returned, but not `model.feature1.layer2`.
    Arguments:
        model (nn.Module): model on which we will extract the features
        return_layers (Dict[name, new_name]): a dict containing the names
            of the modules for which the activations will be returned as
            the key of the dict, and the value of the dict is the name
            of the returned activation (which the user can specify).
    Examples::
        >>> m = torchvision.models.resnet18(pretrained=True)
        >>> # extract layer1 and layer3, giving as names `feat1` and feat2`
        >>> new_m = torchvision.models._utils.IntermediateLayerGetter(m,
        >>>     {'layer1': 'feat1', 'layer3': 'feat2'})
        >>> out = new_m(torch.rand(1, 3, 224, 224))
        >>> print([(k, v.shape) for k, v in out.items()])
        >>>     [('feat1', torch.Size([1, 64, 56, 56])),
        >>>      ('feat2', torch.Size([1, 256, 14, 14]))]
    """
    def __init__(self, model, return_layers):
        if not set(return_layers).issubset([name for name, _ in model.named_children()]):
            raise ValueError("return_layers are not present in model")

        orig_return_layers = return_layers
        return_layers = {k: v for k, v in return_layers.items()}
        layers = OrderedDict()
        for name, module in model.named_children():
            layers[name] = module
            if name in return_layers:
                del return_layers[name]
            if not return_layers:
                break

        super(IntermediateLayerGetter, self).__init__(layers)
        self.return_layers = orig_return_layers
        #self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        out = OrderedDict()
        for name, module in self.named_children():
            x = module(x)
            if name in self.return_layers:
                out_name = self.return_layers[name]
                out[out_name] = x#self.softmax(x)
        return out

In [3]:

class DoubleConv(nn.Module):#为U_Net模型中的双卷积结构
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),#此处包含padding，为了使输出图像与原图像大小相同
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, input):
        return self.conv(input)


class Unet(nn.Module):
    def __init__(self,in_ch,out_ch):
        super(Unet, self).__init__()

        self.conv1 = DoubleConv(in_ch, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        self.conv5 = DoubleConv(512, 1024)
        self.up6 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.conv6 = DoubleConv(1024, 512)
        self.up7 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv7 = DoubleConv(512, 256)
        self.up8 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.conv8 = DoubleConv(256, 128)
        self.up9 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv9 = DoubleConv(128, 64)
        self.conv10 = nn.Conv2d(64,out_ch, 1)

    def forward(self,x):
        c1=self.conv1(x)
        p1=self.pool1(c1)
        c2=self.conv2(p1)
        p2=self.pool2(c2)
        c3=self.conv3(p2)
        p3=self.pool3(c3)
        c4=self.conv4(p3)
        p4=self.pool4(c4)
        c5=self.conv5(p4)
        up_6= self.up6(c5)
        merge6 = torch.cat([up_6, c4], dim=1)
        c6=self.conv6(merge6)
        up_7=self.up7(c6)
        merge7 = torch.cat([up_7, c3], dim=1)
        c7=self.conv7(merge7)
        up_8=self.up8(c7)
        merge8 = torch.cat([up_8, c2], dim=1)
        c8=self.conv8(merge8)
        up_9=self.up9(c8)
        merge9=torch.cat([up_9,c1],dim=1)
        c9=self.conv9(merge9)
        c10=self.conv10(c9)
        out = nn.Sigmoid()(c10)
        return out

model=Unet(1,1).cuda()

In [4]:

#criterion = nn.CrossEntropyLoss()
# model.load_state_dict(torch.load("deep3_(4_8).pkl"))
#criterion = nn.BCELoss()
learning_rate=0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
import csv,os
def get_csv():
    class_path  =  'F:\\Ruiveen2\\Data\\Train_c2\\expert1\\'
    class_path3  = 'F:\\Ruiveen2\\Data\\img\\'
    blanket=set()
    #############case3
#     blanket.add(6)
#     blanket.add(21)
#     blanket.add(27)
#################case1
    blanket.add(8)
    blanket.add(9)
    blanket.add(2)
###############case2
#     blanket.add(5)
#     blanket.add(15)
#     blanket.add(18)
###############case4
#     blanket.add(7)
#     blanket.add(29)
#     blanket.add(34)

#     if 2 in blanket:
#         print("fdsfdsfds")
    with open('my_test.csv','w',newline='')as f:
        f_csv = csv.writer(f)
        i=0
        for img_name in os.listdir(class_path):
            info=str.split(img_name,"_")
            
#             print(img_name)
#             break
           # print(info[0])
            if int(info[0]) in blanket:
                class_path2=class_path+img_name+"\\"
#                 print(class_path2)
#                 break
                for img_name2 in os.listdir(class_path2):
                    img_name3=info[1]+"_"+img_name2
                   # print(img_name3)
                    img_path2= class_path2+ img_name2
                    img_path = class_path3+info[1]+"\\"+ img_name2
#                     print(img_path)
#                     print(img_path2)
#                     break
                    temp=[2]
                    temp.append(img_path)
                    temp.append(img_name3)
                    temp.append(img_path2)
                    temp.append("")
                    f_csv.writerow(temp)
                    i=i+1
                    if i%100==0:
                        print(i)
#                     break
             #   break
get_csv()


100
200
300
400
500
600
700
800
900
1000


In [6]:
#keep the X contain the original image 
def load_data(label_path):
    labels_list=open(label_path, 'r')
    X,Y,Z,W=[],[],[],[]
    i=1
    c=0
    d=0
    for line in labels_list:
        info=str.split(line,",") 
        if i==1:
            print(info[1])
            print(info[3])
        X.append(np.expand_dims(imageio.imread(info[1]),axis=2))#orginal image
#         a=imageio.imread(info[3])
        W.append(np.expand_dims(imageio.imread(info[3]),axis=2))
#         b=np.max(a)
#         if b!=0:
#             b=1
#         if b:
#             d+=1
#         else:
#             c+=1
        Y.append(2)
        Z.append(str(info[2]))
        #break
#         print(X[0].shape)
#         print(X[0].shape)
        i+=1
        if i%100==0:
            print(i)
#     print(c,d)
    return list(zip(X,Y,Z,W))

In [7]:

# data=load_data("/media/students/000FF23E00059C2D/tq/2019/8/20/gen_test2.csv")
# data=load_data2("case4_before_o.csv")
# data=load_data3("my_test2.csv")
# data=load_data("unet_train2.csv")
data=load_data("my_test.csv")

F:\Ruiveen2\Data\img\6522\1.bmp
F:\Ruiveen2\Data\Train_c2\expert1\2_6522\1.bmp
100
200
300
400
500
600
700
800
900
1000


In [8]:
# model.load_state_dict(torch.load("deep1_o.pkl"))
mmpp=87685435454 # 1194110
model.load_state_dict(torch.load("unet1_o.pkl"))

<All keys matched successfully>

In [ ]:
#3.0961676501583497
#11.68

In [10]:
from matplotlib import cm
import cv2

def get_jet():
    colormap_int = np.zeros((256, 3), np.uint8)
    colormap_float = np.zeros((256, 3), np.float)

    for i in range(0, 256, 1):
        colormap_float[255-i, 0] = cm.jet(i)[0]
        colormap_float[255-i, 1] = cm.jet(i)[1]
        colormap_float[255-i, 2] = cm.jet(i)[2]

        colormap_int[255-i, 0] = np.int_(np.round(cm.jet(i)[0] * 255.0))
        colormap_int[255-i, 1] = np.int_(np.round(cm.jet(i)[1] * 255.0))
        colormap_int[255-i, 2] = np.int_(np.round(cm.jet(i)[2] * 255.0))
        #colormap_int = colormap_int[::-1]
    return colormap_int

def print_im(img,img2,_,tar):
    
    img = np.expand_dims(img, 2)
    img = np.concatenate([img,img,img],axis=2)
    c=img2.shape[0]
    d=img2.shape[1]
    heatmap=np.zeros((c,d,3),np.uint8)
    
    for i in range(c):
        for j in range(d):
            heatmap[i][j]=color_map[img2[i][j]]
            
    if tar>1:
        ss=(img2==255).sum()
        if ss>10:
            tar=1
        else: 
            tar=0
            
    heatmap = (cv2.resize(heatmap, dsize=(d,c))*0.5).astype(np.int)
    show_img = np.clip(img + heatmap, a_min=0, a_max=255).astype(np.uint8)
    img = img.astype(np.uint8)
    write_img = np.concatenate((img, show_img), axis=1)
    write_img = cv2.putText(write_img, "target:" + str(tar) , (10, 60), 2,cv2.FONT_HERSHEY_PLAIN, (0, 0, 255))
    cv2.imwrite("unet_test1/" +_, write_img)
    info=str.split(_,".")
    cv2.imwrite("unet_test1/" +str("_")+info[0]+str(".bmp"), img2)


In [ ]:
import random #没有crf
num_epochs=1
miou=0
mcnt=0
with torch.no_grad():
    m=nn.Softmax(dim=1)
    color_map=get_jet()
    batch_size=2
    
    for epoch in range(num_epochs):
        count=0
        #random.shuffle(data)
        X, Y,_,W= zip(*data)

        X=np.array(X).transpose((0,3,1,2))
        W=np.array(W).transpose((0,3,1,2))
        #print(W.shape)
        mm=0
        flag=False
        for i in range(0,X.shape[0],batch_size):
#             print(_[i])
#             if flag==False:
#                 if _[i]!='11607_96.bmp':
#                     continue
#                 else :
#                     flag=True
            if i+batch_size>=X.shape[0]: 
                fff=X.shape[0]-i
                batch_size=fff
                if fff<=1:
                    break
                else:
                    batch=torch.from_numpy(X[i:X.shape[0]]).cuda().float()
                    
            batch=torch.from_numpy(X[i:i+batch_size]).cuda().float()

            outputs=model(batch).squeeze()
#             print("outputs.shape=",outputs.shape)
#             outputs['out']=m(outputs['out'])
#             print("outputs['out'].shape=",outputs['out'].shape)
#             p_,pred=torch.max(outputs['out'],1)#pred中存运行后的结果
#             p_,pred=out
#             print("pred.shape=",pred.shape)
#             print("p_.shape=",p_.shape)
            img2=np.array(outputs.cpu()*255).astype(np.int)#img2 means prediction
#             print(img2.shape)
#             break
           
            img=np.squeeze(batch.cpu().numpy()).astype(np.int)
#             print("img.shape=",img.shape)
#             break
            _o=np.array(_[i:i+batch_size])#图片名
            _y=np.array(Y[i:i+batch_size])#图片名
            _w=np.array(W[i:i+batch_size])#图片名
#             print("_o",_o)
#             print("_y",_y)
#             print("_w",_w)
#             break
            
            for j in range(batch_size):
#                 if _y[j]==0:
#                     continue
                h,w=img[j].shape
                img3=_w[j][0]#img3 means label
                num1=0
                num2=0
                num3=np.sum(img3)
                #print(num3)
                if num3!=0:
                    for p in range(h):
                        for q in range(w):
                            if img2[j][p][q]>0 or img3[p][q]>0:
                                num1+=1
                                if img2[j][p][q]>0 and img3[p][q]>0:
                                    num2+=1
                    miou+=num2/num1
                    mcnt+=1
                    if mcnt>0:
                        print(_o[j],"                          ",num2/num1)
#                 print(img[j].shape)
#                 print(img2[j].shape)
                print_im(img[j],img2[j],_o[j],_y[j])
#                 print(_o[j])
#                 if _o[j]=='14404_101.bmp':
#                     break
                if mcnt>0:
                    if count%128==0:
                        print(count,miou/mcnt)
                count += 1
#         break

6522_100.bmp                            0.6552710313269975
6522_101.bmp                            0.6390568582242137
6522_102.bmp                            0.6189556823168056
6522_103.bmp                            0.5330313325783315
6522_104.bmp                            0.5153474700818812
6522_105.bmp                            0.5314462665736218
6522_106.bmp                            0.5305731612785555
6522_107.bmp                            0.4948710371902571
6522_108.bmp                            0.49280401390783857
6522_109.bmp                            0.44524956118637204
6522_110.bmp                            0.4282722513089005
6522_111.bmp                            0.38937808307593014
6522_112.bmp                            0.35424254687710777
6522_113.bmp                            0.30410996689959546
6522_114.bmp                            0.30298216847714693
6522_115.bmp                            0.2892828364222401
6522_116.bmp                            0.26376952